In [1]:

import sys
import pandas as pd

from keras.applications import xception
from keras.applications import resnet50
from keras.applications import mobilenetv2
# from efficientnet import EfficientNetB3

import time
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
import random

import keras
from keras import regularizers
from keras.models import Sequential
from keras.layers import Conv2D, Dropout, MaxPooling2D, Flatten, Dense, Softmax
from keras.preprocessing import image

from keras.utils import np_utils
from keras.models import load_model
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, K
from keras.models import Input, Model
from sklearn.metrics import f1_score

from keras.preprocessing.image import ImageDataGenerator
import seaborn as sns
import gc

import warnings

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
warnings.filterwarnings('ignore')
# get_ipython().run_line_magic('matplotlib', 'inline')

bKaggle = True
bDebug = False   # train data test.

datadir = './pre/'
modeldir = './pre/'
inputdir='./'
if bKaggle:
    datadir = '../input/carmodel5/ppcarmodel/'
    modeldir = '../input/carmodel6/model/'
    inputdir = '../input/2019-3rd-ml-month-with-kakr/'

print('modeldir=', modeldir)

imagesize=250

def dbgprint(msg):
    if bKaggle:
        os.system('echo "'+msg+'"')
    else:
        print(msg)
dbgprint('hello log')


def new_score(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = (true_positives + K.epsilon()) / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = (true_positives + K.epsilon()) / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

modeldir= ../input/carmodel6/model/


In [3]:
# test generator

if bDebug:
    print('debug: train data verify.')
    dftrain = pd.read_csv(inputdir+'train.csv')
    dftrain['class'] = dftrain['class'].astype('str')
    dftest = dftrain.iloc[8000:,:]
    targetdir = datadir+'train'
else:
    print('test data verify.')
    dftest = pd.read_csv(inputdir+'test.csv')
    targetdir = datadir+'test'

classes = list(str(num) for num in range(1,197))

batch_size=32
datagen = ImageDataGenerator(rescale=1./255)
test_generator = datagen.flow_from_dataframe(dataframe=dftest, 
                                             directory=targetdir,
                                             x_col = 'img_file', 
                                             class_mode = None,
                                             target_size = (imagesize, imagesize),
                                             batch_size=batch_size, shuffle=False )

test data verify.
Found 6150 validated image filenames.


In [4]:
# test 
if True:
    inputs = Input(shape=(imagesize,imagesize,3))
    net = xception.Xception(input_tensor=inputs, input_shape=(imagesize, imagesize, 3), include_top=False, weights='imagenet', pooling='max')
    net2 = Dense(256, activation='relu') (net.layers[-1].output)
    net2 = Dense(196, activation='softmax')(net2)
    model = Model(inputs=inputs, outputs=net2)
    mp = glob.glob(modeldir+'carmodel-v5-1-'+'*')[0]
    print('model:', mp)
    model.load_weights(mp)


model: ../input/carmodel6/model/carmodel-v5-1-xception-129-0.8921.ckpt


In [20]:
from PIL import Image

def load_img_array(filenames):
    nps = []
    for filename in filenames:
        if os.path.exists(datadir+'test/'+filename):
            # load image and flip horizontally
            img = Image.open(datadir+'test/'+filename)
            img = img.resize((imagesize, imagesize))
            img = img.transpose(Image.FLIP_LEFT_RIGHT)
            npimg = np.asarray(img)
            nps.append(npimg)
        else:
            print('not found ', filename)
    return np.asarray(nps)

In [21]:
# test code

bulksize = 2000
idxstart = 0
idxend = bulksize
preds = []
for i in range(len(dftest)//bulksize+1):
    idxstart = i*bulksize
    idxend = (i+1)*bulksize if (i+1)*bulksize < len(dftest) else len(dftest)
    nptest = load_img_array(dftest['img_file'][idxstart:idxend])
    nptest = nptest / 255.0
    print(len(nptest), nptest[0].shape)
    # predict
    pred = model.predict(nptest)
    preds.append(pred)

preds = np.concatenate(preds, axis=0)
print(preds.shape)

1000 (250, 250, 3)
1000 (250, 250, 3)
1000 (250, 250, 3)
1000 (250, 250, 3)
1000 (250, 250, 3)
1000 (250, 250, 3)
150 (250, 250, 3)


In [10]:
# test code

print('predict_generator')
test_generator.reset()
pr = model.predict_generator( test_generator , steps=len(dftest)/batch_size, verbose=1)

print(np.argmax(preds, axis=1))
print(np.argmax(pr, axis=1))

match = np.sum(np.argmax(preds, axis=1)==np.argmax(pr, axis=1))
print(match, '/', len(preds))

predict_generator
193/192 [==============================] - 24s 124ms/step


In [42]:
# model predict
model_cnt=0
if True:
    predictions=[]
    modellist = ['carmodel-v5-1-', 'carmodel-v5-2-', 'carmodel-v5-3-','carmodel-v5-4-', 'carmodel-v5-5-', 'carmodel-v5-6-' ]
#     modellist = ['carmodel-v5-1-']
    for ff, mp in enumerate(modellist):
        files = glob.glob(modeldir+mp+'*')
        if len(files)==0:
            print('not found model:', mp)
            continue
        mp = max(files, key=os.path.getctime)
        msg = '{} model={}'.format(ff, mp)
        dbgprint(msg)
        model_cnt+=1
        
        method='xception'
        if mp.find('resnet50')>0:
            method = 'resnet50'
        elif mp.find('mobilenetv2')>0:
            method = 'mobilenetv2'
        elif mp.find('efficientnetb3')>0:
            method = 'efficientnetb3'
        
        inputs = Input(shape=(imagesize,imagesize,3))
        print('method=', method)
        if method=='xception':
            net = xception.Xception(input_tensor=inputs, input_shape=(imagesize, imagesize, 3), include_top=False, weights='imagenet', pooling='max')
        elif method=='resnet50':
            net = resnet50.ResNet50(input_tensor=inputs, input_shape=(imagesize, imagesize, 3), include_top=False, weights='imagenet', pooling='max')
        elif method=='mobilenetv2':
            net = mobilenetv2.MobileNetV2(input_tensor=inputs, input_shape=(imagesize, imagesize, 3), include_top=False, weights='imagenet', pooling='max')
        elif method=='efficientnetb3':
            net = EfficientNetB3(input_tensor=inputs, input_shape=(imagesize, imagesize, 3), include_top=False, weights='imagenet', pooling='max')
        net2 = Dense(256, activation='relu') (net.layers[-1].output)
        net2 = Dense(196, activation='softmax')(net2)
        model = Model(inputs=inputs, outputs=net2)
        
        print('model',ff+1,':', mp)
        model.load_weights(mp)
        
        print('predict_generator')
        test_generator.reset()
        pr = model.predict_generator( test_generator , steps=len(dftest)/batch_size, verbose=1)
        predictions.append(pr)
        print('prediction',ff+1,':',pr)
        print(pr.shape)
        
        # TTA
        bulksize = 2000
        idxstart = 0
        idxend = bulksize
        preds = []
        for i in range(len(dftest)//bulksize+1):
            idxstart = i*bulksize
            idxend = (i+1)*bulksize if (i+1)*bulksize < len(dftest) else len(dftest)
            nptest = load_img_array(dftest['img_file'][idxstart:idxend])
            nptest = nptest / 255.0
            print(len(nptest), nptest[0].shape)
            # predict
            pred = model.predict(nptest)
            preds.append(pred)

        preds = np.concatenate(preds, axis=0)
        print(preds.shape)
        predictions.append(preds)

#         print('predict')
#         pr2 = model.predict()

method= xception
model 1 : ../input/carmodel6/model/carmodel-v5-1-xception-129-0.8921.ckpt
predict_generator
193/192 [==============================] - 25s 129ms/step
prediction 0 : [[5.0705599e-15 0.0000000e+00 0.0000000e+00 ... 5.8705917e-21
  3.1356052e-30 0.0000000e+00]
 [3.6794965e-23 1.0713371e-32 3.2272692e-19 ... 1.9632371e-15
  9.9999988e-01 3.3179943e-21]
 [0.0000000e+00 7.4608209e-32 5.0946626e-14 ... 9.6230071e-29
  1.9218804e-32 5.9936325e-25]
 ...
 [0.0000000e+00 0.0000000e+00 1.7745896e-32 ... 3.8539011e-24
  3.6936995e-31 1.9371078e-33]
 [1.3905494e-33 0.0000000e+00 9.3787864e-20 ... 3.3457393e-37
  1.6736243e-23 9.7038796e-15]
 [5.7036223e-29 1.0914294e-35 1.5710641e-21 ... 1.3400677e-14
  8.5120734e-27 2.1077964e-25]]
(6150, 196)
2000 (250, 250, 3)
2000 (250, 250, 3)
2000 (250, 250, 3)
150 (250, 250, 3)
(6150, 196)
method= xception
model 2 : ../input/carmodel6/model/carmodel-v5-2-xception-052-0.8709.ckpt
predict_generator
193/192 [==============================] - 25s

In [43]:
print('dftest count=', len(dftest))
print('model count=', model_cnt)
print('model result count=', len(predictions))
print('each result count=', predictions[0].shape)

dftest count= 6150
model count= 6
model result count= 12
each result count= (6150, 196)


In [44]:
if False:
    predictions=[]
    test_generator.reset()
    pr = model.predict_generator( test_generator , steps=len(dftest)/batch_size, verbose=1 )
    # steps=len(dftest)/batchsize
    predictions.append(pr)
    print('prediction',ff,':',pr)
    print(pr.shape)

In [ ]:
# model predict result. how similar? 
model1_pr=np.argmax(predictions[0], axis=1)
model2_pr=np.argmax(predictions[1], axis=1)
model3_pr=np.argmax(predictions[2], axis=1)
print(model1_pr)
print(model2_pr)
print(model3_pr)

# model 3 all matched ratio 
samecnt = np.sum(np.bitwise_and(model1_pr==model2_pr, model1_pr==model3_pr))
print(samecnt, '/', len(model1_pr))

In [ ]:
# ensemble result.....
if True:
    # ensemble~~~~~  mean value. 
    predictions = np.asarray(predictions)
    prk = np.mean(predictions, axis=0 )
    # index class. need to change to label.
    pdi = np.argmax(prk, axis=1)  
    print('ensemble class index:', pdi,  'min=', np.min(pdi), 'max=', np.max(pdi))
    sns.countplot(pdi)

In [ ]:
# class to LABEL...
if True:
    # labeling k=label string, v=class number
#     label = dict((v,k) for k,v in train_generator.class_indices.items())
#     final_pred = [label[k] for k in pdi]
    
    n1=pd.DataFrame(list(range(1,197)), columns=['num']).astype('str')
    n2=n1.sort_values('num').values.squeeze()
    final_pred = [n2[k] for k in pdi]
    
    final_pred = np.asarray(final_pred, dtype=np.int)
    print('final:', final_pred)
    print('count=', len(final_pred))

In [ ]:
# debug result check
if bDebug:
    # evaluation (train data set. we knew the label.)
    reallabel = np.array(dftest['class'].values, dtype=int)
    print(reallabel)
    print( np.sum(final_pred==reallabel), '/', len(final_pred), len(reallabel))    

In [ ]:
# make submission file.
if not bDebug:
    submission = pd.read_csv(inputdir+'sample_submission.csv')
    submission["class"] = final_pred  
    submission.to_csv("submission.csv", index=False)
    print(submission.head())

In [ ]:
if not bDebug:
    sns.countplot(submission["class"], order=submission["class"].value_counts(ascending=True).index)   

In [ ]:
# high rate predictions
dfpredictions = pd.DataFrame(np.max(prk, axis=1).squeeze(), columns=['maxPr'])
dfpredictions.head()

In [ ]:
dfpredictions['maxPr'].plot.hist(bins=20)

In [ ]:
psids = dfpredictions['maxPr']>0.99 
print( np.sum(psids) , '/', len(dfpredictions) )

In [ ]:
if True:
    print('Create Pseudo Label')
    dfpseudo = pd.DataFrame( dftest.loc[psids, 'img_file'] )
    dfpseudo['class'] = final_pred[psids]
    dfpseudo.to_csv('pseudo.csv')

In [ ]:
dfpseudo.head()